In [1]:
from pymongo import MongoClient

client = MongoClient(host="localhost", port=27017)

(list(client.list_databases()))

[{'name': 'admin', 'sizeOnDisk': 40960, 'empty': False},
 {'name': 'config', 'sizeOnDisk': 110592, 'empty': False},
 {'name': 'local', 'sizeOnDisk': 40960, 'empty': False}]